# North Korea provocations database

In [2]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [3]:
import pandas as pd
import altair as alt

In [4]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.set_option("display.max_colwidth", None)

### Download latest version of the database from here

In [5]:
# https://beyondparallel.csis.org/database-north-korean-provocations/

### Import data

In [6]:
src = pd.read_csv("input/Provocations New.csv")

In [8]:
src.columns = (
    src.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=True)
    .str.replace("(", "", regex=True)
    .str.replace(")", "", regex=True)
    .str.replace("-", "_", regex=True)
)

In [19]:
src.sort_values("date", ascending=False)

,date,type,event,description,resources,year,month,day
301,2021-10-19,Missile Provocation,Submarine Launched Ballistic Missile Launch,"North Korea fired a submarine-launched ballistic missile from Sinpo towards the sea between Japan and Korea. The missile was deteced at 10:17am (KST), and traveled about 590 km at a top altitude of 60 km.",https://en.yna.co.kr/view/AEN20211019002958325?section=nk/nk,2021,10,Tuesday
300,2021-09-30,Missile Provocation,Anti-aircraft missile Launch,"According to North Korea's Academy of Defense Science, they test-fired a new type of anti-aircraft missile.",https://en.yna.co.kr/view/AEN20211001000553325?section=nk/nk,2021,9,Thursday
299,2021-09-28,Missile Provocation,Short-range Missile Launch,"North Korea fired a missile at around 6:40am (KST) into the East Sea from Toyang-ri, Jagang Province eastward. KCNA announced the missile as a new ""hypersonic missile"" called Hwasong-8.",https://en.yna.co.kr/view/AEN20210929005552325?section=nk/nk,2021,9,Tuesday
298,2021-09-15,Missile Provocation,Short-range Missile Launch,"North Korea fired two short-range missiles into the East Sea from the central county of Yangdok. They were fired at 12:34 pm and 12:39 pm (KST) respectively, and flew about 800 km (497 miles) at a maximum altitude of around 60 km.",https://en.yna.co.kr/view/AEN20210915005651325,2021,9,Wednesday
297,2021-09-12,Missile Provocation,Long-range Cruise Missile Launch,"North Korea test-fired a new type of long-range cruise missile over two days. According to KCNA, the missile ""travelled for 7,580 seconds along an oval and pattern-8 flight orbits"" and ""hit targets 1,500 km away"" (932 miles).",https://en.yna.co.kr/view/AEN20210913000354325,2021,9,Sunday
296,2021-09-11,Missile Provocation,Long-range Cruise Missile Launch,"North Korea test-fired a new type of long-range cruise missile over two days. According to KCNA, the missile ""travelled for 7,580 seconds along an oval and pattern-8 flight orbits"" and ""hit targets 1,500 km away"" (932 miles).",https://en.yna.co.kr/view/AEN20210913000354325,2021,9,Saturday
295,2021-03-25,Missile Provocation,Short-range Missile Launch,"On March 25, 2021 at 7:06am and 7:25am (KST), North Korea launched two short-range ballistic missiles from the eastern town of Hamju towards the sea between Koea and Japan that flew about 450km (280 miles) at an altitude of 60km (37 miles).",https://en.yna.co.kr/view/AEN20210325000656325?section=nk/nk,2021,3,Thursday
294,2021-03-21,Missile Provocation,Anti-Ship Cruise Missile Launch,"On March 21, 2021 at around 6:36am (KST), North Korea fired two cruise missiles from the western port county of Onchon.",https://en.yna.co.kr/view/AEN20210324000555325?section=news,2021,3,Sunday
293,2020-07-06,Missile Provocation,Anti-Ship Cruise Missile Launch,"On July 7, North Korea launched ship-to-ship missile from a ship in the sea between Korea and Japan that flew about 100 km. South Korean authorities saw the test as part of North Korea's regular maritime exercise.",https://en.yna.co.kr/view/AEN20200730003600325,2020,7,Monday
292,2020-06-16,Other Provocation,Explosion at inter-Korean joint liaison office,"On June 16, 2020, North Korea set off an explosion at the inter-Korean joint liaison office in Kaesong. Three days prior, the sister of North Korean leader Kim Yo-jong had threatened to destroy the “useless” office.",https://en.yna.co.kr/view/AEN20200616008253325?section=nk/nk,2020,6,Tuesday


### Parse dates

In [10]:
src["date"] = pd.to_datetime(src["date"])
src["year"] = src["date"].dt.year
src["month"] = src["date"].dt.month
src["day"] = src["date"].dt.day
src["day"] = src["date"].dt.day_name()

In [13]:
src.head()

,date,type,event,description,resources,year,month,day
0,1958-02-16,Other Provocation,Hijacking of South Korean Plane,North Korean agents hijacked a South Korean plane en route from Busan to Seoul. Twenty-eight kidnapped passengers taken to Pyongyang were eventually released but eight other passengers were never returned.,"https://goo.gl/Xn9Wr4 The US Military Experience in Korea 1871-1982, page 108.",1958,2,Sunday
1,1959-06-15,Other Provocation,NK Fighter Jets Attack U.S. Navy Plane,Two North Korean jet fighters (MiG) attacked and damaged a U.S. navy patrol plane (P4M Mercator) off the eastern coast of North Korea. The pilot returned safely to base in Japan's Honshu Island.,"https://goo.gl/UFh93r The US Military Experience in Korea 1871-1982, page 109.",1959,6,Monday
2,1960-07-30,Other Provocation,Exchange of Fire and Sinking of NK Vessel,A South Korean destroyer sinks a North Korean patrol vessel off the eastern coast killing at least 14 crewmen. This was the first naval clash between the two Koreas since the signing of the Armistice in 1953.,"https://goo.gl/NEgR7x The US Military Experience in Korea 1871-1982, page 109.",1960,7,Saturday
3,1962-11-20,Other Provocation,Exchange of Fire at DMZ,NK troops attacked a UN observation post 7 miles southeast of Panmunjom and an armed clash ensues.,"http://www.cfr.org/content/publications/attachments/Korean_Peninsula_Clashes.pdf; The US Military Experience in Korea 1871-1982, page 113.",1962,11,Tuesday
4,1963-07-29,Other Provocation,Exchange of Fire at DMZ,A group of at least seven North Korean soldiers crossed the DMZ and killed two U.S. soldiers and wounded one during an attack on a jeep with a machine gun and hand grenade. U.S. and South Korean patrols tracked and killed two of the infiltrators.,"https://goo.gl/8PXjzE; The US Military Experience in Korea 1871-1982, page 113.",1963,7,Monday


In [18]:
src.groupby(["type", "year"]).agg({"date": "count"}).reset_index()

,type,year,date
0,Missile Provocation,1984,2
1,Missile Provocation,1985,1
2,Missile Provocation,1990,2
3,Missile Provocation,1991,2
4,Missile Provocation,1992,1
5,Missile Provocation,1993,2
6,Missile Provocation,1994,2
7,Missile Provocation,1995,1
8,Missile Provocation,1997,1
9,Missile Provocation,1998,1


In [15]:
missiles = src[src["type"] == "Missile Provocation"]

In [16]:
missiles.sort_values("date", ascending=False).head()

,date,type,event,description,resources,year,month,day
301,2021-10-19,Missile Provocation,Submarine Launched Ballistic Missile Launch,"North Korea fired a submarine-launched ballistic missile from Sinpo towards the sea between Japan and Korea. The missile was deteced at 10:17am (KST), and traveled about 590 km at a top altitude of 60 km.",https://en.yna.co.kr/view/AEN20211019002958325?section=nk/nk,2021,10,Tuesday
300,2021-09-30,Missile Provocation,Anti-aircraft missile Launch,"According to North Korea's Academy of Defense Science, they test-fired a new type of anti-aircraft missile.",https://en.yna.co.kr/view/AEN20211001000553325?section=nk/nk,2021,9,Thursday
299,2021-09-28,Missile Provocation,Short-range Missile Launch,"North Korea fired a missile at around 6:40am (KST) into the East Sea from Toyang-ri, Jagang Province eastward. KCNA announced the missile as a new ""hypersonic missile"" called Hwasong-8.",https://en.yna.co.kr/view/AEN20210929005552325?section=nk/nk,2021,9,Tuesday
298,2021-09-15,Missile Provocation,Short-range Missile Launch,"North Korea fired two short-range missiles into the East Sea from the central county of Yangdok. They were fired at 12:34 pm and 12:39 pm (KST) respectively, and flew about 800 km (497 miles) at a maximum altitude of around 60 km.",https://en.yna.co.kr/view/AEN20210915005651325,2021,9,Wednesday
297,2021-09-12,Missile Provocation,Long-range Cruise Missile Launch,"North Korea test-fired a new type of long-range cruise missile over two days. According to KCNA, the missile ""travelled for 7,580 seconds along an oval and pattern-8 flight orbits"" and ""hit targets 1,500 km away"" (932 miles).",https://en.yna.co.kr/view/AEN20210913000354325,2021,9,Sunday


In [17]:
missiles.groupby(["event"]).agg({"date": "count"}).reset_index()

,event,date
0,Anti-Ship Cruise Missile Launch,18
1,Anti-aircraft missile Launch,1
2,Ballistic Missile Launch (type unknown),2
3,Inter-Continental Ballistic Missile/Short-Range Missile Launches,1
4,Inter-continental Ballistic Missile Launch,2
5,Intermediate-range Ballistic Missile Launch,12
6,Joint Missile Test with China,1
7,Long-range Cruise Missile Launch,2
8,Long-range Surface-to-Air Missile Launch,1
9,Medium-range Ballistic Missile Launch,13
